In [1]:
import functools
import geopy
from matplotlib import collections as mc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyproj
import requests
import scipy as sp
import rtree
import seaborn as sb
from scipy import signal
# import shapely
import shapely.geometry
%pylab inline

import data_munging

Populating the interactive namespace from numpy and matplotlib


/home/zblan/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
rides, readings = data_munging.read_raw_data()
readings = data_munging.clean_readings(readings)
readings = data_munging.add_proj_to_readings(readings, data_munging.NAD83)

In [3]:
print rides.shape
print readings.shape
n, p = readings.shape

(94, 7)
(35884, 39)


In [8]:
nearest_request = 'http://127.0.0.1:5000/nearest?loc={0},{1}'
map_request = 'http://127.0.0.1:5000/match?loc={0},{1}&t={2}&loc={3},{4}&t={5}'

In [18]:
map_request

'http://127.0.0.1:5000/match?loc={0},{1}&t={2}&loc={3},{4}&t={5}'

In [6]:
a_reading = readings.loc[0, :]

In [13]:
# This does not work because OSRM does not accept floats as times. 
test_map_request = map_request.format(*tuple(a_reading[['start_lat', 'start_lon', 'start_time',
                                                'end_lat', 'end_lon', 'end_time']]))

In [20]:
test_map_request = map_request.format(a_reading['start_lat'],
                                      a_reading['start_lon'], 
                                      0,
                                      a_reading['end_lat'],
                                      a_reading['end_lon'],
                                      1)

In [21]:
print test_map_request
print requests.get(test_map_request).json()

http://127.0.0.1:5000/match?loc=41.8967525317,-87.6342008241&t=0&loc=41.8967114871,-87.6342004788&t=1
{u'status': 200, u'status_message': u'Found matchings', u'matchings': [{u'geometry': u'}qd|nA|pwcfDpAC', u'route_summary': {u'total_time': 1, u'total_distance': 5}, u'matched_points': [[41.896751, -87.634207], [41.89671, -87.634205]], u'hint_data': {u'checksum': 428354038, u'locations': [u'_____6TIBQDRAwAAKwAAALQAAAAAAAAAAAAAAP____8RAAAAL0t_AuHOxvoAAAEB', u'_____6TIBQDRAwAAIAAAAL8AAAAAAAAAAAAAAP____8RAAAABkt_AuPOxvoAAAEB']}, u'indices': [0, 1], u'matched_names': [u'North Wells Street', u'North Wells Street']}]}
